In [1]:
import numpy as np
import pandas as pd
import os, re, shutil
import ast

In [2]:
import sys 
sys.path.append('Shell_Code_Modified/')
import Farm_Evaluator_Vec

In [3]:
#initialize all the variables
# wind farm size, cells
cand_per_run=100

folderName="Ver1_N/2007"
ExtName= (folderName+"/results/sg")

best_layout_prefix="sugga_best_generation_layout_N"
best_energy_prefix="sugga_best_generation_eta_N"
default_location='../data/Shell_Hackathon_Dataset/'
Wind_data_year=['2017','2015','2014','2013','2009','2008','2007','avg']

output_folder_location=folderName+"/summary"
if not os.path.exists(output_folder_location):
    os.makedirs(output_folder_location)
### Read Experiment Spec
fname=folderName+"/"+'exp_specs.txt'
file = open(fname, 'r')
f = file.read()
file.close()
paraDic = ast.literal_eval(f)
cols_cells = int(paraDic['cols_cells'])  # number of cells each row
rows_cells = int(paraDic['rows_cells'])  # number of cells each column
cell_width = int(paraDic['cell_width'])
wt_N=int(paraDic['wt_N'])

In [4]:
def convert_xy_pos(pop,rows,cell_width,N_tur):
    xy_position = np.zeros((N_tur,2), dtype=np.float32)
    curr_tur=0
    for ind in range(rows * rows):
        if pop[ind] == 1:
            r_i = np.floor(ind / rows)
            c_i = np.floor(ind - r_i * rows)
            xy_position[curr_tur,0] = c_i * cell_width + cell_width/2
            xy_position[curr_tur,1] = r_i * cell_width + cell_width/2
            curr_tur +=1
    return (xy_position)
            

In [5]:

# Create regex with prefix + number + extension.
fileRegex = re.compile(r'(%s)\d' % best_layout_prefix)
    
# Make a list to contain the file with prefix.
fileNames = []
for filename in os.listdir(ExtName):
    #print(filename)
    if re.search(fileRegex, filename):
        fileNames.append(filename)
# Make sure the fileName in the list have a right order.

run_results_dict={}

result_summary_df=pd.DataFrame() #dataframe to analyze results
result_list_all={}  #complete summary with xy coordinates position.
dict_index=0

result_layout_all=None
best_eta_layout_all=None

for curr_run_filename in fileNames:
    print("Processing File:"+curr_run_filename)
    filePath=ExtName+"/"+curr_run_filename
    result_layout=np.genfromtxt(filePath, delimiter="  ", dtype=np.int32)
#     print(result_layout_all.shape)
#     print(result_layout.shape)

    
    best_eta_filePath=curr_run_filename.replace(best_layout_prefix,best_energy_prefix)
    best_eta_filePath = ExtName+"/"+best_eta_filePath
    best_eta_layout=np.genfromtxt(best_eta_filePath, delimiter="  ")
#     print(best_eta_layout.shape)
#     print(best_eta_layout_all.shape)
    if result_layout_all is None:
        result_layout_all=result_layout
        best_eta_layout_all=best_eta_layout
    else:
        result_layout_all=np.vstack((result_layout,result_layout_all))
        best_eta_layout_all=np.append(best_eta_layout,best_eta_layout_all,axis=0)

print(result_layout_all.shape)
print(best_eta_layout_all.shape)

best_run_sorts=np.argsort(best_eta_layout_all)
#   print(best_eta_layout)
total_cands=best_run_sorts.shape[0]
dict_index=0
for curr_cand in range(cand_per_run):

    curr_run_results={}
    
    tur_xy_coord=convert_xy_pos(result_layout_all[best_run_sorts[total_cands-1-curr_cand]],rows_cells,cell_width,wt_N)

    #         print(tur_xy_coord.shape)
    for curr_wind_inst in Wind_data_year:
        wind_inst_freq_file=default_location+"Wind_Data/wind_data_"+(curr_wind_inst)+".csv"
        total_energy, turbine_power=Farm_Evaluator_Vec.get_AEP_results(default_location=default_location,turb_coords=tur_xy_coord,wind_inst_freq_file=wind_inst_freq_file)
        curr_run_results[str(curr_wind_inst)]=total_energy
#             print("Year: "+str(curr_wind_inst)+" Energy: "+ str(total_energy))
#         print(curr_run_results)
    curr_run_results['dict_index']=dict_index
#           print(curr_run_results)
#         print(result_summary_df)

    result_summary_df=result_summary_df.append(pd.DataFrame.from_records([curr_run_results]),ignore_index = True)
    curr_run_results['XY_coords']=tur_xy_coord
    result_list_all[dict_index]=curr_run_results
    dict_index +=1


Processing File:sugga_best_generation_layout_N50_1_20201013003246.dat
Processing File:sugga_best_generation_layout_N50_0_20201012222518.dat
(400, 6400)
(400,)


In [6]:
#assign overall score based on year. 
result_summary_df['best_score']=(result_summary_df['2007']+result_summary_df['2008']+result_summary_df['2009']+result_summary_df['2013']+result_summary_df['2014']+result_summary_df['2015']+result_summary_df['2017'])/7
result_summary_df.sort_values(by=['best_score'],inplace=True,ascending=False)
result_summary_df['file_name']=np.arange(0,result_summary_df.shape[0])
result_summary_df.head()
#can be used to select top N (future in case needed)

,2017,2015,2014,2013,2009,2008,2007,avg,dict_index,best_score,file_name
4,567.437902,581.646703,468.585614,493.371351,478.080315,521.260902,527.784720,519.542045,4,519.738215,0
0,567.241011,581.495326,468.315841,493.282930,478.141334,521.338529,527.896399,519.475445,0,519.673053,1
1,567.154996,581.567439,468.398581,493.236682,478.049906,521.206299,527.826424,519.437617,1,519.634332,2
3,567.084487,581.440188,468.346751,493.197584,478.040349,521.160953,527.809783,519.385822,3,519.582871,3
2,567.149048,581.397816,468.333385,492.964602,478.089104,521.128037,527.823617,519.357350,2,519.555087,4


In [7]:
#write outputs
os.makedirs(output_folder_location, exist_ok=True)
outfile_index=0
for index, curr_row in result_summary_df.iterrows():
    curr_dictIndex=curr_row['dict_index']
    curr_result_xy=pd.DataFrame(result_list_all[curr_dictIndex]['XY_coords'])
    curr_result_xy.columns=["x","y"]
    curr_result_xy.to_csv(output_folder_location+"/best"+str(curr_row['file_name'])+".csv",index=False)
    #outfile_index +=1

result_summary_df.to_csv(output_folder_location+"/result_summary.csv",index=False)
print("All Files are stored at location: "+output_folder_location)

All Files are stored at location: Ver1_N/2007/summary
